In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import json
import seaborn as sns
from datetime import datetime
import re
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.feature_selection import f_classif, mutual_info_classif

# Any results you write to the current directory are saved as output.
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
train = pd.read_csv('/kaggle/input/recommendationsv4/train.csv', low_memory=False)
test = pd.read_csv('/kaggle/input/recommendationsv4/test.csv', low_memory=False)
submission = pd.read_csv('/kaggle/input/recommendationsv4/sample_submission.csv', low_memory=False)

In [4]:
test.info()

In [5]:
train.info()

In [6]:
#Преобразуем json с отзывами в Dataframe
with open('/kaggle/input/recommendationsv4/meta_Grocery_and_Gourmet_Food.json') as f:
    meta_from_json_list = []
    for line in f.readlines():
        meta_from_json_list.append(json.loads(line))
        
meta_df = pd.DataFrame(meta_from_json_list)

In [7]:
meta_df.info()

In [8]:
#Видим, что датасет с отзывами и тренировочный датасет можно объединить по полю asin
train_meta = pd.merge(train, meta_df, on='asin')

In [9]:
train_meta.info()

In [10]:
#Создадим массивы под разные типы столбцов
num_cols = []
binary_cols = []
categorial_cols = []

In [11]:
#Скопируем датасет для анализа
df = train_meta.copy()

In [12]:
print(df.overall.value_counts())
print(df.overall.describe())
print("Пропусков: ", df.overall.isnull().sum())
df.overall.value_counts().plot.barh()

In [13]:
#Категориальный признак с целыми числами без пропусков
df['overall']=df['overall'].astype('int64')
categorial_cols.append('overall')

In [14]:
print(df.verified.value_counts())
print(df.verified.describe())
print("Пропусков: ", df.verified.isnull().sum())
sns.countplot(x = 'verified', data = df)

In [15]:
#Бинарный признак. Преобразуем к числовому представлению
df['verified'] = df['verified'].map(lambda x: 1 if x == True else 0).astype('int64')
binary_cols.append('verified')

In [16]:
#Видим абсолютную корреляцию двух столбцов и оставляем timestamp
print(df[['reviewTime', 'unixReviewTime']].head())
df = df.drop(['reviewTime'], axis=1)

In [17]:
#Оставим только год отзыва
df['unixReviewTime'] = df['unixReviewTime'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y')).astype('int64')

In [18]:
df['unixReviewTime'] = df['unixReviewTime'].astype('int64')
df['unixReviewTime'].describe()

In [19]:
num_cols.append('unixReviewTime')

In [20]:
print(df.asin.value_counts())
print(df.asin.describe())
print("Пропусков :", df.asin.isnull().sum())

In [21]:
#Этот стобец был нужен для соспоставлния датасетов. Поскольку он содержит слишком много уникальных значений, то его можно удалить
df = df.drop(['asin'], axis=1)

In [22]:
print(df.reviewerName.value_counts())
print(df.reviewerName.describe())
print("Пропусков: ", df.reviewerName.isnull().sum())

In [23]:
#Опять видим слишком много уникальных значений, что не позволяет использовать признак как категориальный.
#Можно было бы попробовать использовать параметр реального имени или имени по-умолчанию (Amazon Customer, Kindle Customer), но поскольку визуально
#таких имен меньше 5%, то такое разделение также не выглядит разумным. Удаляем.
df = df.drop(['reviewerName'], axis=1)

In [24]:
#Сравним два столбца. В summary резюмируется содержание reviewText. Оставим его для дальнейшего анализа
df[['reviewText', 'summary']]

In [25]:
df = df.drop(['reviewText'], axis=1)

In [26]:
print(df.summary.value_counts())
print(df.summary.describe())
print("Пропусков: ", df.summary.isnull().sum())

In [27]:
#Преобразуем к нижнему регистру и заполняем пропуски
df['summary_low'] = df.summary.str.lower()
df.summary_low.fillna('')

In [28]:
#Проверяем статистически значимые слова на выборке. Видим, что помимо звезд, часто встречаются слова 'great' и 'good' 
df.head(10000).summary_low.apply(lambda x: pd.value_counts(str(x).split(" "))).sum(axis = 0).nlargest(10)

In [29]:
#Преобразуем все значения к полученному списку
star_list = ['five stars', 'four stars', 'three stars', 'two stars', 'one star', 'great', 'good']
for stars in star_list:
    df['summary'] = np.where(df['summary_low'].str.contains(stars), stars, df['summary_low'])
df['summary'] = np.where(df['summary'].isin(star_list), df['summary'], 'No info')

In [30]:
df = df.drop(['summary_low'], axis=1)
categorial_cols.append('summary')

In [31]:
print(df.vote.value_counts())
print(df.vote.describe())
print("Пропусков: ", df.vote.isnull().sum())

In [32]:
#Слишком много пропусков. Удаляем
df = df.drop(['vote'], axis=1)

In [33]:
print(df['style'].value_counts())
print(df['style'].describe())
print("Пропусков: ", df['style'].isnull().sum())

In [34]:
#Слишком много пропусков, а остальные данные не имеют выраженной группировки. Удаляем
df = df.drop(['style'], axis=1)

In [35]:
print(df.image_x.value_counts())
print(df.image_x.describe())
print("Пропусков: ", df.image_x.isnull().sum())

In [36]:
#Слишком много пропусков. Удаляем
df = df.drop(['image_x'], axis=1)

In [37]:
print(df.userid.value_counts())
print(df.userid.describe())
print("Пропусков: ", df.userid.isnull().sum())

In [38]:
num_cols.append('userid')

In [39]:
print(df.itemid.value_counts())
print(df.itemid.describe())
print("Пропусков: ", df.itemid.isnull().sum())

In [40]:
num_cols.append('itemid')

In [41]:
print(df.rating.value_counts())
print(df.rating.describe())
print("Пропусков: ", df.rating.isnull().sum())

In [42]:
num_cols.append('rating')

In [43]:
#Удаляем эти признаки, как не имеющие четко выраженной группировки
df = df.drop(['category'], axis=1)
df = df.drop(['description'], axis=1)
df = df.drop(['title'], axis=1)
df = df.drop(['brand'], axis=1)
df = df.drop(['rank'], axis=1)

In [44]:
print(df.main_cat.value_counts())
print("Пропусков: ", df.main_cat.isnull().sum())

In [45]:
#Преобразуем все значения к популярному списку
categories_list = ['Grocery', 'Health & Personal Care', 'Amazon Home', 'All Beauty', 'Sports & Outdoors', 'Industrial & Scientific', 'Office Products']
for cats in categories_list:
    df['main_cat'] = np.where(df['main_cat'].str.contains(cats), cats, df['main_cat'])
df['main_cat'] = np.where(df['main_cat'].isin(categories_list), df['main_cat'], 'Other')

In [46]:
print(df.price.value_counts())
print(df.price.describe())
print("Пропусков: ", df.price.isnull().sum())

In [47]:
#Выделим все, что преобразуется в число
df['price'] = df['price'].astype('str')
df['price'] = df['price'].apply(lambda x: re.sub('\D', '', x))
df['price'] = df['price'].apply(lambda x: 0 if x == '' else x)
df['price'] = df['price'].astype('float')
df['price'] = df['price'].apply(lambda x: x/100)

In [48]:
#На боксплоте видно, что много выбросов в большую сторону
sns.boxplot(df['price'])

In [49]:
#Посмотрим на квантили и отсечем по примерному разумному максимуму
df['price'].quantile(0.9, interpolation="midpoint")
df['price'] = df['price'].apply(lambda x: 0 if x > 30 else x)

In [50]:
#Заполним нули средним значением
mean_price = df['price'].mean()
df['price'] = df['price'].apply(lambda x: mean_price if x == 0 else x)
num_cols.append('price')

In [51]:
#Анализ следующих признаков также не дал понимания как их категоризировать для обучения, поэтому удаляем
df = df.drop(['also_buy'], axis=1)
df = df.drop(['also_view'], axis=1)

In [52]:

df['image_y'] = df['image_y'].fillna(0)
df['image_y']= df['image_y'].apply(lambda x: 0 if x==0 else 1)

In [53]:
binary_cols.append('image_y')

In [54]:
#Эти признаки удаляем как неинформативные
df = df.drop(['date', 'feature', 'details', 'similar_item', 'tech1', 'fit'], axis=1)

In [55]:
print(categorial_cols)
print(num_cols)
print(binary_cols)

In [56]:
# преобразование с помощью LabelEncoder()
label_encoder = LabelEncoder()
# преобразуем категории времени в числовые
mapped = pd.Series(label_encoder.fit_transform(df['unixReviewTime']))
print(dict(enumerate(label_encoder.classes_)))

In [57]:
df['unixReviewTime'] = label_encoder.fit_transform(df['unixReviewTime'])
categorial_cols.append('unixReviewTime')

In [58]:
mapped = pd.Series(label_encoder.fit_transform(df['main_cat']))
print(dict(enumerate(label_encoder.classes_)))


In [59]:
df['main_cat'] = label_encoder.fit_transform(df['main_cat'])
categorial_cols.append('main_cat')

In [60]:
#Посмотрим на тепловую карту корреляций и увидим высокую корреляцию признаков overall и rating
plt.figure(figsize=(10,5))
sns.heatmap(df.corr(), annot=True)

In [61]:
#Удалим признак overall
df = df.drop(['overall'], axis = 1)
binary_cols.append('rating')

In [62]:
#На боксплотах никаких аномалий не замечено
for col in ['unixReviewTime', 'userid', 'itemid', 'price']:
    plt.title(f"{col}")
    sns.boxplot(x='rating', y=col, data=df)
    plt.show()

In [63]:
#Бинарные признаки выглядят полезными
from itertools import combinations
from scipy.stats import ttest_ind
def get_stat_dif(column):
    cols = df.loc[:, column].value_counts().index[:10]
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        if ttest_ind(df.loc[df.loc[:, column] == comb[0], 'rating'], 
                        df.loc[df.loc[:, column] == comb[1], 'rating']).pvalue \
            <= 0.05/len(combinations_all): 
            print('Найдены статистически значимые различия для колонки', column)
            break
for col in binary_cols:
    get_stat_dif(col)

**Построение модели**

In [64]:
import scipy.sparse as sparse
from sklearn import preprocessing
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
import sklearn
from sklearn.model_selection import train_test_split

import scipy.sparse as sparse
from sklearn.metrics import auc, roc_auc_score, roc_curve

In [65]:
train_data, test_data = train_test_split(train,random_state=32, shuffle=True)

In [66]:
ratings_coo = sparse.coo_matrix((train_data['rating'].astype(int),
                                 (train_data['userid'],
                                  train_data['itemid'])))

In [132]:
NUM_THREADS = 4
NUM_COMPONENTS = 80
NUM_EPOCHS = 20
RANDOM_STATE = 32


model = LightFM(learning_rate=0.07, loss='logistic',
                no_components=NUM_COMPONENTS, random_state = RANDOM_STATE)
model = model.fit(ratings_coo, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)

In [133]:
def roc_auc_curve(y_true, y_pred_prob):
    fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
    plt.figure()
    plt.plot([0, 1], label='Случайный классификатор', linestyle='--')
    plt.plot(fpr, tpr, label = 'LightFM')
    plt.title('ROC AUC = %0.3f' % roc_auc_score(y_true, y_pred_prob))
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.legend(loc = 'lower right')
    plt.show()

In [134]:
#Получаем предсказание:
preds = model.predict(test_data.userid.values,
                      test_data.itemid.values)
#Считаем roc_auc_score
sklearn.metrics.roc_auc_score(test_data.rating,preds)
#Cтроим ROC AUС
roc_auc_curve(test_data.rating,preds)
normalized_preds = (preds - preds.min())/(preds - preds.min()).max()


In [135]:
ratings_coo1 = sparse.coo_matrix((train['rating'].astype(int),
                                 (train['userid'],
                                  train['itemid'])))
model = LightFM(learning_rate=0.1, loss='logistic',
                no_components=NUM_COMPONENTS, random_state = RANDOM_STATE)

model1 = model.fit(ratings_coo1, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)

#Получаем предсказание:
preds1 = model1.predict(test.userid.values, test.itemid.values)

In [136]:
normalized_preds1 = (preds1 - preds1.min())/(preds1 - preds1.min()).max()


In [137]:
test['pred_rating'] = normalized_preds1
test

In [138]:
train_data, test_data = train_test_split(df,random_state=32, shuffle=True)
ratings_coo2 = sparse.coo_matrix((train_data['rating'].astype(int),
                                 (train_data['userid'],
                                  train_data['itemid'])))

model = LightFM(learning_rate=0.07, loss='logistic',
                no_components=NUM_COMPONENTS, random_state = RANDOM_STATE)
model2 = model.fit(ratings_coo2, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)

In [139]:
#Получаем предсказание:
preds2 = model2.predict(test_data.userid.values, test_data.itemid.values)

In [140]:
# Подсчитываем метрику roc_auc_score
sklearn.metrics.roc_auc_score(test_data.rating,preds2)
#Cтроим ROC AUС и наблюдаем небольшое улучшение
roc_auc_curve(test_data.rating,preds2)

In [141]:
preds_sub = model2.predict(test.userid.values, test.itemid.values)

In [142]:
normalized_preds_sub = (preds_sub - preds_sub.min())/(preds_sub - preds_sub.min()).max()

In [143]:
#Создаем матрицу для продуктов
item_features = train_data[['image_y', 'main_cat', 'price']]
norm_ifeatures = (item_features - item_features.mean()) / item_features.std()
item_features=(sparse.csr_matrix(norm_ifeatures)).astype(np.float32)

In [144]:
#и клиентов
user_features = train_data[['verified','unixReviewTime']]
norm_ufeatures = (user_features - user_features.mean()) / user_features.std()
user_features = (sparse.csr_matrix(norm_ufeatures)).astype(np.float32)

In [145]:
NUM_THREADS = 4
NUM_COMPONENTS = 80
NUM_EPOCHS = 20
RANDOM_STATE = 32
model = LightFM(learning_rate=0.07, loss='logistic',
                no_components=NUM_COMPONENTS)
model_feat = model.fit(ratings_coo, user_features=user_features, item_features=item_features, epochs=NUM_EPOCHS,
                       num_threads=NUM_THREADS)

In [147]:
userid = np.array(test_data.userid).astype(np.int32)
itemid =np.array(test_data.itemid).astype(np.int32) 

In [148]:
#Получаем предсказание:
preds0 = model_feat.predict(userid, itemid, item_features=item_features,
                            user_features=user_features,num_threads=NUM_THREADS)

In [149]:
sklearn.metrics.roc_auc_score(test_data.rating,preds0)

In [150]:
#Эмбеддинги

item_biases, item_embeddings = model2.get_item_representations()
item_biases.shape, item_embeddings.shape

In [85]:
#Используем метод knn (ближайших соседей)

In [88]:
!pip install nmslib

In [151]:
import nmslib
 
#Создаём граф для поиска и добавляем товары
nms_idx = nmslib.init(method='hnsw', space='cosinesimil')
nms_idx.addDataPointBatch(item_embeddings)
nms_idx.createIndex(print_progress=True)

In [152]:
def nearest_item_nms(itemid, index, n=10):
    nn = index.knnQuery(item_embeddings[itemid], k=n)
    return nn

In [153]:
mapper = dict(zip(meta_df['asin'],meta_df['title']))

In [154]:
train['title'] = train.asin.apply(lambda x: mapper[x])
test['title'] = test.asin.apply(lambda x: mapper[x])
prod_df = train.drop(['verified','reviewTime','reviewerName','reviewText','summary','unixReviewTime','vote','style','image'],axis=1)

In [94]:
prod_df.head(5)

In [155]:
#Ищем товары, похожие на  id=17322
nbm = nearest_item_nms(17322,nms_idx)[0]

prod_df[prod_df.itemid.isin(nbm)]

In [156]:

submission['rating']= normalized_preds_sub
submission

In [157]:
submission.to_csv('submission_log.csv', index=False)